In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as st

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

import suppression as s
import utils
import linear_model

In [ ]:
gaba_fn = 'gaba_data.txt'
supp_fn = 'supp_data_individual_20170427.txt'

In [ ]:
sdf = utils.load_psychophys(supp_fn)
gdf = utils.load_gaba(gaba_fn)

In [ ]:
plot_dir = "plots/redo-201807"

## Analyze tasks separately (before subsetting to include common subjects)##

In [ ]:
pp_subjs = np.unique(sdf.Subject)
n_pp_subjs = len(pp_subjs)
gaba_subjs = np.unique(gdf.subjName)
n_gaba_subjs = len(gaba_subjs)
print(f"Psychophysics subjects (n={n_pp_subjs}):\n", pp_subjs)
print(f"GABA subjects (n={n_gaba_subjs}):\n", gaba_subjs)

# GABA only analyses

### GABA t-test, CON v AMB

In [ ]:
pop_group = gdf.groupby("Population")
pop_group.describe()

In [ ]:
gaba_per_group = [col for col_name, col in pop_group['GABA']]
(tstat, pval) = st.ttest_ind(*gaba_per_group)
print(tstat, pval)

** Thus we find no significant difference in GABA levels between CON and AMB. **

### GABA violin plot, all subjects

In [ ]:
with s.PdfPages(f"{plot_dir}/gaba_diffs_n{n_gaba_subjs}.pdf") as pdf:
    fig = plt.figure(figsize=(6,8))  # create a figure object
    ax = fig.add_subplot(1, 1, 1)
    ax = sns.violinplot(y='GABA',x='Presentation',hue='Population',data=gdf,split=True,inner='stick',ax=ax)
    ax.xaxis.set_visible(False)
    ax.set_ylabel("GABA (relative to Creatine)")
    plt.show(ax.figure)
    pdf.savefig(ax.figure)
    plt.close(ax.figure)
    plt.close('all')

# Select one psychophysical task's data #

In [ ]:
task = 'SS'
sdf = sdf[sdf['Task']==task]

## Find the RelMaskContrast at which NDE and DE most different within population using t-tests

### New way, moving the old way to functions in utils.py, 7-12-18

In [ ]:
gvars_test = ['Task','Orientation','Presentation','Population']
# equal_var=False makes it Welch's t-test, which does not assume the grooups have equal variance
df_to_model = utils.find_xvalue_to_predict(sdf, gvars_test, test_func=st.ttest_ind, equal_var=False)

In [ ]:
# The line below shows data that confirms that each bin has a different RelMaskContrast at its center for NDE and DE
# This is bad so we're going to redo the binning, ignoring Eunice's BinNumbers and instead discretizing the linear model fit
#df_to_model.groupby(gvars_test + ["Eye", "BinNumber"])[["RelMCToPred","BinCenterRelMaskContrast"]].describe()

In [ ]:
df_to_model.columns

### Add a field that is the observed ThreshElev data point in the Bin to predict

*** Actually, this isn't possible. Most subjects have fewer data points than there are bins, so often will not have an observation in the critical bin. Keep using the predicted ThreshElev instead. ***

In [ ]:
n_pp_subjs_thistask = len(np.unique(df_to_model.Subject))
n_amb_subjs_thistask = len(np.unique((df_to_model[df_to_model['Population']=='Amblyope'])['Subject']))
print(f"There are {n_pp_subjs_thistask} subjects for Task {task}, of which {n_amb_subjs_thistask} are Amblyopes.")

# Modeling

### Begin grouping data into conditions to model Subject's ThreshElev as a function of logRelContrast #

In [ ]:
pp_gvars = ['Task','Orientation','Presentation','Population','Subject','Eye','Trace'] # One condition
pp_gvars_base = pp_gvars + ['BaselineThresh']

In [ ]:
groups = df_to_model.groupby(pp_gvars) 

### Linear model (defined in linear_model.py, uses lmfit)###

In [ ]:
# initialize parameters
lm_params = linear_model.parameters()
preds_lm = groups.apply(utils.model_threshold, linear_model.err, linear_model.thresh, lm_params, ret='preds')

## Descriptive statistics

** Note on column names: **
 * RelMaskContrast is the presented MaskContrast (absolute, in C%) divided by the subject's BaselineThresh
 * ThreshElev is the *observed* threshold elevation at that RelMaskContrast
 * ThreshPred is the *model's predicted* threshold elevation at that RelMaskContrast
 * RelMCToPred is the RelMaskContrast that is the center of the bin with the biggest NDE/DE difference (BinNumberToPred)

In [ ]:
preds_lm.groupby(pp_gvars)[["RelMaskContrast", "ThreshElev", "ThreshPred"]].describe(percentiles=[.5])

In [ ]:
s.group_facet_plots(preds_lm, s.subject_fit_plot,
                    f"{plot_dir}/{task}_regressions_combinedplots_n{n_pp_subjs_thistask}_TOP_welch.pdf",
                    ['Task','Orientation','Presentation'], #each combo of this gets its own page
                    row='Population',col='Eye',# facet rows and columns
                    x="RelMaskContrast", y="ThreshElev", # x, y
                    hue="Subject",yerr='ThreshElev_SE',fmt_obs='.',fmt_pred='x:',Ycol="ThreshPred") 

In [ ]:
s.group_facet_plots(preds_lm, s.population_fit_plot,
                    f"{plot_dir}/{task}_regressions_combinedplots_n{n_pp_subjs_thistask}_TO_welch.pdf",
                    ['Task','Orientation'], #each combo of this gets its own page
                    row='Presentation',col='Eye',# facet rows and columns
                    x="RelMaskContrast", y="ThreshElev", # x, y
                    hue="Population",yerr='ThreshElev_SE',fmt_obs='.',fmt_pred='x:',Ycol="ThreshPred") 

### Model the data again, but this time return parameters, not predictions ###

In [ ]:
print(pp_gvars_base)
pp_gvars_base_mcpred = pp_gvars_base + ['RelMCToPred']
groups_with_baseline_mcpred = df_to_model.groupby(pp_gvars_base_mcpred)

In [ ]:
pfit = groups_with_baseline_mcpred.apply(utils.model_threshold, linear_model.err, linear_model.thresh, lm_params, ret='weights').reset_index()

In [ ]:
pfit.head()

In [ ]:
pfit.groupby(["Orientation", "Presentation", "Population", "Eye", "RelMCToPred"]).describe()

In [ ]:
pfit['ThreshPredCritical'] = pfit['y_int'] + pfit['slope'] * pfit['RelMCToPred']
pfit['ThreshPredCriticalUnnorm'] = pfit['ThreshPredCritical'] * pfit['BaselineThresh']

### Depth of suppression measures ("how far from ThreshElev=1?")

In [ ]:
pfit['DepthOfSuppressionPred'] = (-1) + pfit['ThreshPredCritical']

In [ ]:
# melt the result of the modeling into long format for plotting
pfit_all_ppsub = pd.melt(pfit, id_vars=pp_gvars, var_name='measure')
pfit_all_ppsub.head()

## Subset to include only (GABA and psychophyics) subjects

In [ ]:
gaba_and_pp_subjs = list(np.intersect1d(pp_subjs, gaba_subjs))
n_gaba_and_pp_subjs = len(gaba_and_pp_subjs)

In [ ]:
sdf = sdf[sdf.Subject.isin(gaba_and_pp_subjs)] # only subjects who did _the current_ pp task and GABA
gaba_and_pp_subjs_thistask = np.unique(sdf.Subject)
n_gaba_and_pp_subjs_thistask = len(gaba_and_pp_subjs_thistask)
print(f"Of the {n_gaba_and_pp_subjs} subjects with both GABA and psychophysics data, {n_gaba_and_pp_subjs_thistask} have both for task {task}.")

In [ ]:
gdf = gdf[gdf.subjName.isin(sdf.Subject)] # only subjects who did both tasks
print(f"Of the {len(gdf)} subjects with GABA and {task} data, {len(gdf[gdf.Population=='Amblyope'])} are Amblyopes.")
n_this_task = len(gdf)

## Combine Psychophysics and GABA below

In [ ]:
#Grab the GABA measure for each subject and append it to each observation for easy plotting
comb = pfit_all_ppsub.join(gdf.set_index(['subjName'])['GABA'], on=['Subject'])
print(len(comb))

#subset to include only those subjects with GABA data
comb_gabappsub = comb[~np.isnan(comb['GABA'])]
print(len(comb_gabappsub), f"{len(np.unique(comb_gabappsub.Subject))} subjects")

In [ ]:
comb_gabappsub.head(n=10)

In [ ]:
np.unique(comb_gabappsub.measure)

In [ ]:
#graphs!
with s.PdfPages(f"{plot_dir}/gaba_vs_{task}_n{n_this_task}_linear.pdf") as pdf:
    plot_groups = comb_gabappsub.groupby(['Task','Orientation','measure'])
    for gv, gr in plot_groups:
        print(gv, np.all(np.isnan(gr['value'])))
        #print(gr.columns)
        #if gv[0]=='SS' and gv[1]=='Cross': continue
        g2 = s.gaba_vs_psychophys_plot_2line(gv, gr)
        pdf.savefig(g2.fig)
        #g4 = s.gaba_vs_psychophys_plot_4line(gv, gr)
        #pdf.savefig(g4.fig)
        
    plt.close('all')

#### Conclusions from the graphs we just generated (SS):
 * BaselineThresh
    * There is a negative relationship between GABA and BaselineThresh for both eyes, both populations, both surround conditions. This is an interesting finding in itself...
      * (nDicho and nMono are identical since this is baseline thresh, ie just one eye)
 * DepthOfSuppression
    * In SS/Cross/Dicho, more GABA = less suppression for NDE, but = more suppression for DE. This is especially true of Amblyopes, while in Controls the effect is weak/insignificant.
    * In SS/Iso/Dicho, more GABA = less suppression for NDE, = more suppression for DE. Effect is very strong for Amblyopes and absent for other conditions(AMB/Mono and all Controls)
 * Critical Bin Center (RelMCToPred)
    * In SS/Cross/Dicho the NDE has a higher (numerically, in multiples of baseline) RelMCToPred for both AMB and CON
    * But in SS/Cross/Mono it's the reverse, DE has higher RelMCToPred for both populations
 * ThreshPredCritical
    * For both Cross and Iso, Dicho/AMB shows biggest difference b/t NDE and DE in predicted threshold elevation. Other conditions show little to no difference.
 * ThreshPredCriticalUnnorm
    * When we normalize by each eye's baseline, this effect is still present.
 * Linear model slope
    * AMB/Dicho shows biggest difference between trends in each eye for both presentation conditions. more GABA ~ lower slope of the line for NDE, while DE is basically flat.
 * Linear model y-int
    * AMB/Dicho has opposite GABA vs y-int slopes for both Iso and Cross, but the distributions overlap for all conditions and populations, so this is a weak finding.

### Combine measures across the two eyes

 * Does it make sense to combine all measures across both eyes (i.e. by subtracting?) For example, ThreshElev is in units of baseline, and the baseline varies by eye. So perhaps only a few measures should be combined -- say, slope/yint, ThreshPredCriticalUnnorm. 

In [ ]:
#subset to include only these measures:
# - baseline threshold
# - unnormalized predicted threshold elevation at critical bin center
# - slope of ThreshElev ~ RelMaskContrast line
# - y-int of this line
measures = comb_gabappsub[comb_gabappsub["measure"].isin(["BaselineThresh","ThreshPredCriticalUnnorm","slope","y_int"])]
#print(len(comb_all_ppsub))

In [ ]:
np.unique(measures.measure)

In [ ]:
paired_obs = measures.groupby(['Task', 'Orientation', 'Population', 'Presentation', 'Subject', 'measure'])

def get_eyediff_value(g):
    if len(g)==2: # this will exclude paired observations where there was no data for one eye
        value_diff = g[g['Eye']=='Nde'].value.iat[0] - g[g['Eye']=='De'].value.iat[0]
        #print(g.name, value_diff)
        return pd.Series([value_diff], ['Nde-De'])

In [ ]:
obs_diff = paired_obs.apply(get_eyediff_value).reset_index()

In [ ]:
obs_diff.head(n=6)

In [ ]:
comb_botheyes = obs_diff.join(gdf.set_index(['subjName'])['GABA'], on=['Subject'])

In [ ]:
comb_botheyes.head()

In [ ]:
print(len(np.unique(comb_botheyes.Subject)))

In [ ]:
#graphs!
with s.PdfPages(f"{plot_dir}/gaba_vs_{task}_combeyes_n{n_this_task}.pdf") as pdf:
    plot_groups = comb_botheyes.groupby(['Task','Orientation','measure'])
    for gv, gr in plot_groups:
        print(gv)
        g2 = s.gaba_vs_psychophys_plot_2line_2eye(gv, gr)
        pdf.savefig(g2.fig)
        
    plt.close('all')

In [ ]:
# more graphs for presentation!
with s.PdfPages(f"{plot_dir}/gaba_vs_{task}_combeyes_n{n_this_task}_condensed.pdf") as pdf:
    plot_groups = comb_botheyes.groupby(['Task','measure'])
    for gv, gr in plot_groups:
        print(gv)
        if gv[-1] not in ["BaselineThresh", "RelMCToPred"]: # use this line to exclude measures we don't want
            g2 = s.gaba_vs_psychophys_plot_2line_2eye(gv, gr, row="Orientation", size=8, aspect=1.2)
            pdf.savefig(g2.fig)
        
    plt.close('all')

### Orientation-selective suppression can go here

 * We should start off by doing each eye separately, right? So, starting from comb_gabappsub df.
 * Or if we started from the combined-eye data generated above, use measures df.

In [ ]:
oss_gvars = ["Task", "Presentation", "Population", "Subject", "Eye", "Trace", "measure", "GABA"]

In [ ]:
def calculate_orientation_selective_suppression(df, **kwargs):
    #print(df[['Orientation', 'value']])
    iso_cross_oss_ratio = df[df.Orientation=='Iso']['value'].iloc[0]/df[df.Orientation=='Cross']['value'].iloc[0]
    print(f"Iso/Cross ratio: {iso_cross_oss_ratio}")
    return pd.Series(iso_cross_oss_ratio, ['value'])
    #print(df[df['Orientation'=='Cross']]) #/df[df['Orientation'=='Cross']]['value']

In [ ]:
oss_df = comb_gabappsub.groupby(oss_gvars).apply(calculate_orientation_selective_suppression).reset_index()

In [ ]:
oss_df.head()

In [ ]:
oss_df.columns

In [ ]:
#graphs!
with s.PdfPages(f"{plot_dir}/gaba_vs_{task}_oss_n{n_this_task}.pdf") as pdf:
    plot_groups = oss_df.groupby(['Task','measure'])
    for gv, gr in plot_groups:
        if gv[-1] not in ["BaselineThresh", "RelMCToPred"]:
            print(gv, np.all(np.isnan(gr['value'])))
            g2 = s.gaba_vs_oss_plot_2line(gv, gr)
            pdf.savefig(g2.fig)
        
    plt.close('all')

#### Conclusions from these graphs:
 * The two eyes seem to not be that different in their Iso/Cross ratios of the various measures.